In [1]:
import numpy as np
import nibabel as nib
import os

import matplotlib.pyplot as plt
import seaborn as sns

from pylab import rcParams

from sklearn.preprocessing import scale
import pandas as pd

from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import KFold

from sklearn.metrics import mean_absolute_error as MAE

def make_age_list(files, age_df):
    age_list = []
    for file in files:
        ID = file[17:25]
        age_list.append(df.loc[df[0] == ID][1].values[0])
    return age_list

In [10]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.xception import Xception

from keras.models import Model 

In [34]:
def preprocess_data(data, k):
    new_data = np.zeros((data.shape[0], 121, 121, 3))
    for i in range(3):
        new_data[...,0] = data[:,:,k,:]
    new_data /= np.max(new_data)
    return new_data
    

In [39]:
new_data = preprocess_data(data,60)
new_data.shape

(645, 121, 121, 3)

In [2]:
#reading white matter and age files

files = os.listdir('wm_data')
df = pd.read_csv('original_participant_data.tsv', header=None, sep='\t')
ages = make_age_list(files,df)

data = np.zeros((len(files), 121,145,121))
for sub in range(len(files)):
    img = nib.load('wm_data/'+files[sub])
    data[sub,...] = img.get_data()

In [61]:
xception = Xception(include_top=False, weights='imagenet', 
                        input_tensor=None, input_shape=(121,121,3), pooling=None, classes=1000)

In [ ]:
act_layers = []
shapes = []
for layer in xception.layers:
    if layer.name[-3:] == 'act':
        act_layers.append(layer.name)
        shapes.append(np.prod(layer.output.get_shape().as_list()[1:]))
        
features = []

feature_data = np.zeros((645, 26, 46592))
counter = 0
for i in tqdm(range(len(shapes))):
    if shapes[i] == 46592:
        model = Model(inputs=xception.input, outputs=xception.get_layer(act_layers[i]).output)
        feature_data[:,counter,:] = model.predict(new_data).reshape(645, 46592)
        counter += 1